Part 2: Supervised Learning Model
Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
import json
import pandas as pd
import numpy as np
from joblib import load, dump
from sklearn.ensemble import AdaBoostClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from helpers import kaggle_submission
from process_and_ml.where_is import WhereIs
from process_and_ml import load_test_file, load_customers, CleanUp
from process_and_ml import fs_pipeline, confirm_equal_columns_dataframe
from process_and_ml.pipeline import compute_metrics
from process_and_ml.train import TrainAfterPipeline


In [2]:
where_is = WhereIs(False)


In [3]:
all_paths = where_is.get_paths_list

In [4]:
all_paths

['/home/jair/Documents/arvato_project/process_and_ml/../data/costumer_data/Udacity_AZDIAS_052018.csv',
 '/home/jair/Documents/arvato_project/process_and_ml/../data/costumer_data/Udacity_CUSTOMERS_052018.csv',
 '/home/jair/Documents/arvato_project/process_and_ml/../data/cleaned_data/customers_clean.csv',
 '/home/jair/Documents/arvato_project/process_and_ml/../data/cleaned_data/test_catboost.csv',
 '/home/jair/Documents/arvato_project/process_and_ml/../data/cleaned_data/train_cleaned.csv',
 '/home/jair/Documents/arvato_project/process_and_ml/../data/cleaned_data/test_correct.csv',
 '/home/jair/Documents/arvato_project/process_and_ml/../data/cleaned_data/azdias_clean.csv',
 '/home/jair/Documents/arvato_project/process_and_ml/../data/submissions/catboost_all_features.csv',
 '/home/jair/Documents/arvato_project/process_and_ml/../data/submissions/catboost_first_shoot.csv',
 '/home/jair/Documents/arvato_project/process_and_ml/../data/submissions/catboost_all_features_weights_2.csv',
 '/home/j

In [13]:
train_df = pd.read_csv(all_paths[20], sep=';')

/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
train_df = train_df.rename(columns={'CAMEO_INTL_2015': 'CAMEO_DEUINTL_2015'})

In [15]:
response = train_df['RESPONSE']

In [16]:
train_df.drop(['RESPONSE'], axis=1, inplace=True)


In [17]:
customer_df = load_customers(all_paths[1])

In [18]:
confirm_equal_columns_dataframe(train_df, customer_df)

set()

In [19]:
train_df.CAMEO_DEUG_2015.tail()

42957    3.0
42958    6.0
42959    4.0
42960    8.0
42961    4.0
Name: CAMEO_DEUG_2015, dtype: object

In [20]:
df_test, lnr_for_kaggle = load_test_file(all_paths[19])

In [21]:
clean_up = CleanUp(all_paths)

In [30]:
dfs_dict = clean_up.pipeline_clean_up(dfs={'train':train_df, 'customers': customer_df, 'test': df_test},unknowns_df=all_paths[18])


                      Unnamed: 0.1  Value  TestDvC
Attribute                                         
AGER_TYP                         1     -1        2
ALTERSKATEGORIE_GROB             6  -1, 0        2
ANREDE_KZ                       34  -1, 0        2
BALLRAUM                        41     -1        2
BIP_FLAG                        49     -1        2


/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [31]:
dfs_dict['train']['CAMEO_DEUINTL_2015'].head()

0    34.0
1    32.0
2    14.0
3    14.0
4    41.0
Name: CAMEO_DEUINTL_2015, dtype: object

In [32]:
train_df_clean = dfs_dict['train']

In [33]:
train_df_clean.loc[:, 'CAMEO_DEUG_2015'] = train_df_clean.CAMEO_DEUG_2015.astype('float64')


/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [34]:
customer_df.CAMEO_DEUG_2015.dtypes

dtype('float64')

In [35]:
train_df_clean.CAMEO_DEUG_2015.dtypes

dtype('float64')

In [36]:
train_df_clean.loc[:, 'CAMEO_DEUG_2015'] = train_df_clean.CAMEO_DEUG_2015.astype('float64')

/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [37]:
train_df_clean.CAMEO_DEUINTL_2015.dtypes

dtype('O')

In [38]:
dfs_dict['customers'].CAMEO_DEUINTL_2015.dtypes

dtype('float64')

In [39]:
columns_list = [col for col in train_df.columns if col.startswith('CAMEO')]

In [40]:
train__df_after_pipe = fs_pipeline(train_df_clean)

/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/p

Created WEALTH and LIFE_AGE column from 'CAMEO_DEUINTL_2015', original column was droped
Created 'PLZ8_BAUMAX_BUSSINESS' and 'PLZ8_BAUMAX_FAMILY' from 'PLZ8_BAUMAX_FAMILY', original column was droped.
Created 'PRAEGENDE_JUGENDJAHRE_GEN' and 'PRAEGENDE_JUGENDJAHRE_MOV' from 'PRAEGENDE_JUGENDJAHRE', original column was droped.
Working on Binary columns.
Shape Before Remove KBA -> (42962, 276)
Removing KBA05 columns
Get_Dummies with pandas
Get_Dummies with pandas cat column = ['CAMEO_DEU_2015']


In [41]:
with open('../data/cleaned_data/azdias_columns.json', 'r') as jsonfile:
    azdias_columns = json.load(jsonfile)


In [42]:
train__df_after_pipe = train__df_after_pipe[azdias_columns['columns']]

In [43]:
unsupervised_pipeline = load('../models/unsupervised_transform.joblib')

/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator SimpleImputer from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator Pipeline from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/jair/.cache/pypoetry/virtualenvs/arvatocapst

In [44]:
train__df_after_pipe = unsupervised_pipeline.transform(train__df_after_pipe)

In [45]:
test__df_after_pipe = fs_pipeline(dfs_dict['test'])

/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/p

Created WEALTH and LIFE_AGE column from 'CAMEO_DEUINTL_2015', original column was droped
Created 'PLZ8_BAUMAX_BUSSINESS' and 'PLZ8_BAUMAX_FAMILY' from 'PLZ8_BAUMAX_FAMILY', original column was droped.
Created 'PRAEGENDE_JUGENDJAHRE_GEN' and 'PRAEGENDE_JUGENDJAHRE_MOV' from 'PRAEGENDE_JUGENDJAHRE', original column was droped.
Working on Binary columns.
Shape Before Remove KBA -> (42833, 276)
Removing KBA05 columns
Get_Dummies with pandas
Get_Dummies with pandas cat column = ['CAMEO_DEU_2015']


/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/pandas/core/series.py:4515: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [46]:
test__df_after_pipe = test__df_after_pipe[azdias_columns['columns']]


In [47]:

test__df_after_pipe = unsupervised_pipeline.transform(test__df_after_pipe)

In [48]:
# Let's Start the experiments

In [49]:
train_class = TrainAfterPipeline(train__df_after_pipe, response)

In [50]:
# First the simple model as a base for us

In [51]:
log_reg = LogisticRegression()


In [52]:
# As wrote on proposal let's start with the more simple model. LogisticRegressionCV

In [53]:
logistic_params_grid = {'solver': ['newton-cg', 'lbfgs', 'liblinear'],
                   'penalty': ['l2'],
                   'C': [100, 10, 1.0, 0.1, 0.01]}


In [54]:
lg_result = train_class.train_grid_search(model=log_reg, grid=logistic_params_grid)

Fitting 60 folds for each of 15 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   17.8s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:   34.2s
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:   57.6s
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:  1.2min finished


0.5226542309843866
LogisticRegression(C=10, solver='liblinear')


In [57]:
lg_result.best_params_

{'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}

In [58]:
compute_metrics(lg_result.best_estimator_, train_class.X_test, train_class.y_test)

Metrics(ACC=0.9866552874544186, AUC=0.5298357719295358)

In [79]:
log_tags = {'algo':'logisticRegression',
        'datasets':'train',
        'sample':1,
        'cols':'unsupervised_learning',
        'rows':train__df_after_pipe.shape[0]}

log_experiment_name = 'LogisticRegression'

log_final_param = {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
train_class.train(LogisticRegression,
                  run_name='logictic_sc',
                  experiment_name=log_experiment_name,
                  params=log_final_param,
                  tags=log_tags,
                  data={'x':train__df_after_pipe, 'y': response.values})

LogisticRegression(solver='liblinear')

In [ ]:
# And now Let's try AdaBoost with DecisionTree

In [59]:
ada_model = AdaBoostClassifier(DecisionTreeClassifier())


In [60]:
ada_params_grid = {'learning_rate': [0.1,0.2],
              'n_estimators':[30, 50, 75, 100],
              'random_state': [42],
              'base_estimator__max_depth': [1, 3],
              'base_estimator__max_features': [20, None]}

In [61]:
ada_result = train_class.train_grid_search(model=ada_model, grid=ada_params_grid)

Fitting 60 folds for each of 32 candidates, totalling 1920 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 852 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 1640 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 1920 out of 1920 | elapsed:  9.6min finished


0.542608204278298
AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.1, n_estimators=30, random_state=42)


In [62]:
ada_result.best_params_

{'base_estimator__max_depth': 1,
 'base_estimator__max_features': None,
 'learning_rate': 0.1,
 'n_estimators': 30,
 'random_state': 42}

In [63]:
ada_tags = {'algo':'AdaBoost',
        'datasets':'train',
        'sample':1,
        'cols':'unsupervised_learning_8',
        'rows':train__df_after_pipe.shape[0]}

ada_experiment_name = 'AdaBoost'

ada_final_param = {
 'learning_rate': 0.2,
 'n_estimators': 30,
 'random_state': 42}
model = train_class.train(AdaBoostClassifier, run_name='ada_sec', experiment_name=ada_experiment_name, params=ada_final_param, tags=ada_tags, data={'x':train__df_after_pipe,'y':response.values})

In [64]:
compute_metrics(lg_result.best_estimator_, train_class.X_test, train_class.y_test)


Metrics(ACC=0.9866552874544186, AUC=0.5298357719295358)

Part 3: Kaggle Competition
Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link here, you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [65]:
kaggle_submission(
    column_lnr=lnr_for_kaggle,
    y_pred=model.predict_proba(test__df_after_pipe)[:, 1],
    submission_filename='adaboost_hyper_opt',
   submission_message='AdaBoost GridSearch 75 estimators, unsupervised pipeline'
)



100%|██████████| 1.06M/1.06M [00:02<00:00, 424kB/s] 


Successfully submitted to Udacity+Arvato: Identify Customer Segments


In [ ]:
## Final experiment, just for fun

In [27]:
from hpsklearn import HyperoptEstimator, any_classifier, any_preprocessing

from hyperopt import tpe

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [31]:
estim = HyperoptEstimator(classifier=any_classifier('my_clf'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=80,
                          trial_timeout=300)

In [32]:
estim.fit(x_train, y_train)

 50%|█████     | 1/2 [00:00<?, ?trial/s, best loss=?]

/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



[07:33:03] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 89%|████████▉ | 8/9 [00:00<?, ?trial/s, best loss=?]

/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



[07:33:13] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 94%|█████████▍| 15/16 [00:00<?, ?trial/s, best loss=?]

...

/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



[08:03:11] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 99%|█████████▊| 75/76 [00:00<?, ?trial/s, best loss=?]

/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



[08:04:18] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
100%|██████████| 80/80 [00:00<00:00,  1.20trial/s, best loss: 0.010972568578553665]
[08:04:37] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [33]:
print(estim.best_model())


{'learner': XGBClassifier(base_score=0.5, booster='gbtree',
              colsample_bylevel=0.7582060475754435, colsample_bynode=1,
              colsample_bytree=0.8002680366839026, gamma=0.00019873598617675767,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.013018747690767048, max_delta_step=0, max_depth=1,
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=1, num_parallel_tree=1, random_state=0,
              reg_alpha=0.0036346131696351493, reg_lambda=1.5327239321644701,
              scale_pos_weight=1, seed=0, subsample=0.8198683518149491,
              tree_method='exact', validate_parameters=1, verbosity=None), 'preprocs': (MinMaxScaler(feature_range=(-1.0, 1.0)),), 'ex_preprocs': ()}


In [36]:
compute_metrics(estim.best_model()['learner'], x_test, y_test)

Metrics(ACC=0.9866552874544186, AUC=0.5)

In [39]:
estim.best_model()['learner'].save_model('../models/xgboost.json')

/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/xgboost/sklearn.py:537: UserWarning: kwargs is not saved in Scikit-Learn meta.
  warnings.warn(str(k) + ' is not saved in Scikit-Learn meta.', UserWarning)


In [47]:
#Time to run And post on kaggle Competition
test__df_after_pipe = fs_pipeline(dfs_dict['test'])


/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/p

Created WEALTH and LIFE_AGE column from 'CAMEO_DEUINTL_2015', original column was droped
Created 'PLZ8_BAUMAX_BUSSINESS' and 'PLZ8_BAUMAX_FAMILY' from 'PLZ8_BAUMAX_FAMILY', original column was droped.
Created 'PRAEGENDE_JUGENDJAHRE_GEN' and 'PRAEGENDE_JUGENDJAHRE_MOV' from 'PRAEGENDE_JUGENDJAHRE', original column was droped.
Working on Binary columns.
Shape Before Remove KBA -> (42833, 276)
Removing KBA05 columns
Get_Dummies with pandas
Get_Dummies with pandas cat column = ['CAMEO_DEU_2015']


/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/pandas/core/series.py:4515: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [48]:
test__df_after_pipe = test__df_after_pipe[azdias_columns['columns']]

In [49]:
test__df_after_pipe = unsupervised_pipeline.transform(test__df_after_pipe)

In [50]:
test__df_after_pipe.shape


(42833, 4)

In [53]:
kaggle_submission(
    column_lnr=lnr_for_kaggle,
    y_pred=estim.best_model()['learner'].predict_proba(test__df_after_pipe)[:, 1],
    submission_filename='xgboost_hyper_opt',
   submission_message='Xgboost bayesian hyper [200 estimators, 3 depth] 1.0 sample, unsupervised pipeline'
)




100%|██████████| 706k/706k [00:02<00:00, 316kB/s]  


Successfully submitted to Udacity+Arvato: Identify Customer Segments


In [ ]:
# Conclusion, the best result we reach is was with Catboost, after we did unsupervised learning we catch results bellow the random walk.

